# Fine Tuning mit Unloth AI

Quelle: https://colab.research.google.com/drive/1VYkncZMfGFkeCEgN2IzbZIKEDkyQuJAS?usp=sharing#scrollTo=rkp0uMrNpYaW



In [ ]:
import os
from unsloth import FastLanguageModel, to_sharegpt, standardize_sharegpt
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import TextStreamer
import torch
import subprocess
import time


In [ ]:

# **1. Setup und Vorbereitung**
# Installiere die notwendigen Bibliotheken und richte die Umgebung ein.
# Hier wird Unsloth installiert, eine Bibliothek zur Optimierung von Sprachmodellen.
!pip install unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


# Jeremy Modell anpassen

In [ ]:

# **2. Modell Laden**
# Lade ein vortrainiertes Sprachmodell und seinen zugehörigen Tokenizer.
# `max_seq_length`: Maximale Sequenzlänge, die das Modell verarbeiten kann.
# `load_in_4bit`: Aktiviert die Verwendung von 4-Bit-Quantisierung, um Speicherplatz zu sparen.
# `dtype`: Datentyp für die Berechnung, hier wird der Standard des Modells verwendet.

max_seq_length = 2048
load_in_4bit = True
dtype = None  # Automatische Erkennung basierend auf Hardware und Modell

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",  # Ersetze durch das gewünschte vortrainierte Modell
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


# Jeremy Datensatz einbinden vorgabe siehe 4.

In [ ]:

# **3. Vorbereitung der eigenen Daten**
# Lade den Datensatz, der für das Finetuning verwendet werden soll.
# Hier wird ein CSV-Dateiformat erwartet. Passe den Dateinamen und Split entsprechend an.
data_file = "deine_datei.csv"  # Pfad zur Datei
dataset = load_dataset(
    "csv",              # Dateityp (z. B. CSV oder Excel)
    data_files=data_file,  # Dateipfad
    split="train"       # Trainingssplit laden
)


# Datensatz struktur

In [ ]:

# **4. Datentransformation**
# Konvertiere den Datensatz in ein ShareGPT-ähnliches Format, das für Chatmodelle geeignet ist.
# Passe die Namen der Eingabe- und Zielspalten sowie das Format des Prompts an deine Datenstruktur an.
merged_prompt = """[[Die Eingabe ist {input_col}.]]
                   [[Erwartete Ausgabe ist {output_col}.]]"""  # Definiert die Struktur des Prompts
output_column_name = "output_col"  # Spaltenname für die erwartete Ausgabe

# Wende die Transformation auf den Datensatz an
dataset = to_sharegpt(
    dataset,
    merged_prompt=merged_prompt,  # Das definierte Prompt-Format
    output_column_name=output_column_name,  # Zielspalte
    conversation_extension=5,  # Optional: Anzahl zusätzlicher Konversationsturns
)
dataset = standardize_sharegpt(dataset)  # Standardisiere das Format des Datensatzes


In [ ]:

# **5. Training Konfigurieren und Starten**
# Konfiguriere die Trainingsparameter wie Batch-Größe, Lernrate und maximale Schritte.
training_args = TrainingArguments(
    per_device_train_batch_size=2,  # Anzahl der Trainingsinstanzen pro Gerät und Schritt
    gradient_accumulation_steps=4,  # Schritte, bevor die Gradienten aktualisiert werden
    warmup_steps=5,  # Anzahl der Warmup-Schritte für die Lernrate
    max_steps=60,  # Maximale Anzahl an Trainingsschritten
    learning_rate=2e-4,  # Anfangslernrate
    fp16=True,  # Mixed Precision Training für schnellere Berechnungen
    logging_steps=1,  # Wie oft Logs geschrieben werden (in Schritten)
    output_dir="outputs",  # Speicherort für Modell, Checkpoints und Logs
    seed=42,  # Zufallssaat für reproduzierbare Ergebnisse
    optim="adamw_8bit",  # Optimierer, der für 8-Bit-Gewichte geeignet ist
)

# Erstelle den Trainer, um das Modell zu finetunen
trainer = SFTTrainer(
    model=model,  # Vortrainiertes Modell
    tokenizer=tokenizer,  # Passender Tokenizer
    train_dataset=dataset,  # Der vorbereitete Datensatz
    dataset_text_field="text",  # Feldname für den Text im Datensatz
    max_seq_length=max_seq_length,  # Maximale Sequenzlänge
    dataset_num_proc=2,  # Anzahl paralleler Prozesse für Datenvorverarbeitung
    args=training_args,  # Trainingsparameter
)

# Starte das Training
trainer.train()


In [ ]:

# **6. Modell Speichern**
# Speichere das finetunete Modell und den zugehörigen Tokenizer in einem Verzeichnis.
save_dir = "finetuned_model"
model.save_pretrained(save_dir)  # Speichere das Modell
tokenizer.save_pretrained(save_dir)  # Speichere den Tokenizer


In [ ]:

# **7. Export für Ollama**
# Exportiere das Modell im GGUF-Format, um es in Ollama verwenden zu können.
# Wähle die gewünschte Quantisierungsmethode (z. B. `q8_0` für 8-Bit-Quantisierung).
quantization_method = "q8_0"  # Optionen: "q4_k_m", "q5_k_m", "f16"
model.save_pretrained_gguf(save_dir, tokenizer, quantization_method=quantization_method)


In [ ]:

# **8. Ollama Server Starten**
# Starte den Ollama-Server, um das Modell lokal verfügbar zu machen.
subprocess.Popen(["ollama", "serve"])  # Startet den Server-Prozess
time.sleep(3)  # Warte, bis der Server vollständig gestartet ist


In [ ]:

# **9. Modelfile Generieren und Modell in Ollama Importieren**
# Generiere eine Modellkonfigurationsdatei für Ollama und importiere das Modell.
ollama_modelfile = tokenizer._ollama_modelfile  # Hole die Modelfile-Information
with open(os.path.join(save_dir, "Modelfile"), "w") as f:
    f.write(ollama_modelfile)  # Schreibe die Datei

# Importiere das Modell in Ollama
subprocess.run(["ollama", "create", "unsloth_model", "-f", os.path.join(save_dir, "Modelfile")])


In [ ]:

# **10. Inferenz in Ollama**
# Sende eine Anfrage an den Ollama-Server, um das Modell zu testen.
messages = [
    {
        "role": "user",
        "content": "Die Eingabe ist Beispieltext. Erwartete Ausgabe ist Beispielantwort."
    }
]

# Sende die Anfrage an den Ollama-Server
response = subprocess.run([
    "curl",
    "http://localhost:11434/api/chat",
    "-d",
    f"""
    {{
        "model": "unsloth_model",
        "messages": {messages}
    }}
    """
], capture_output=True)

# Ausgabe der Antwort des Servers
print(response.stdout.decode("utf-8"))